In [39]:
import numpy as np
from itertools import product
from more_itertools import run_length
import time

In [78]:
def vocabulary(letters, l):
    aux = [letters]*l
    return [''.join(k) for k in list(product(*aux))]
    

def simplification_rules(L):

    rules = {}

    if letters == ['H','T']:

        # H rules
        for n in range(1,L+1):
            rules[f'H{n}'] = ('' if n%2==0 else 'H')

        # T rules
        rules['T1'] = 'T'
        rules['T2'] = 'S'
        rules['T4'] = 'Z'
        rules['T8'] = ''
        rules['T3'] = 'ST'
        rules['T5'] = 'ZT'
        rules['T6'] = 'ZS'
        rules['T7'] = 'ZST'
        for n in range(9,Lmax+1):
            rules[f'T{n}'] = rules[f'T{n-8}']

        # S rules
        rules['S1'] = 'S'
        rules['S2'] = 'Z'
        rules['S3'] = 'ZS'
        rules['S4'] = ''
        for n in range(5,Lmax+1):
                    rules[f'S{n}'] = rules[f'S{n-4}']

        # Z rules
        rules['Z1'] = 'Z'
        rules['Z2'] = ''
        for n in range(3,Lmax+1):
                    rules[f'Z{n}'] = rules[f'Z{n-2}']        

        # conjugation rules
        rules['HZH'] = 'X'
        rules['HSH'] = 'Sd'
        rules['HSdH'] = 'S'
        rules['HTH'] = 'Td'
        rules['HTdH'] = 'T'
        
        return rules

    else:
        print('Rules not implemented!')
        
        
def reduced_vocabulary(letters, Lmax): 
    
    maxnsimp = 100 
    
    vocab = set([])
    simp = {}
    totalsize = np.sum([len(letters)**l for l in range(1,Lmax+1)])

    print(f'There are {totalsize} words in the vocabulary with length up to {Lmax}')

    # build vocab with a first round of simplification
    for l in range(1,Lmax+1):
        for k,v in simplification_rules(l).items(): 
            simp[k] = v
        vocab_l = vocabulary(letters, l)
        for word in vocab_l:
            word = list(run_length.encode(word))
            word = ' '.join([k[0]+str(k[1]) for k in word])
            # print(f'Vocabulary using run length encoding:\n{vocab}\n')
            word = word.split(' ')
            word = [(simp[syl] if syl != '' else '') for syl in word]
            word = ''.join(word)
            vocab.add(word)

    size = len(vocab) # will be updated    
    print(f'\n---- simplification #{1}: reduced to {size} unique words')

    # few more rounds of simplification
    for i in range(2,maxnsimp):

        vocab = [list(run_length.encode(word)) for word in vocab]
        vocab = [' '.join([k[0]+str(k[1]) for k in word]) for word in vocab]
        # print(f'Vocabulary using run length encoding:\n{vocab}\n')
        vocab = [word.split(' ') for word in vocab]
        vocab = [[(simp[syl] if syl != '' else '') for syl in word] for word in vocab]
        vocab = list(set([''.join(word) for word in vocab]))
        print(f'\n---- simplification #{i}: reduced to {len(vocab)} unique words')
        if len(vocab) < size:
            size = len(vocab)
        else:
            print(f'\nDONE!\n\nCompressed to {round(100*size/totalsize,2)}% of the original vocabulary\n')
            break
    
    return vocab

In [80]:
letters = ['H','T']
Lmax = 15

start = time.time()
vocab = reduced_vocabulary(letters, Lmax)
end = time.time()
print(f'Total time: {round(end-start,2)}s\n')

print(f'Here are a few of the {len(vocab)} unique words:\n\n{vocab[:20]}')    

vocab_cliff = [w for w in vocab if w.count('T')==0]
print(f'\nFrom these, {len(vocab_cliff)} are Cliffords:\n')
print(vocab_cliff)

There are 65534 words in the vocabulary with length up to 15

---- simplification #1: reduced to 12649 unique words

---- simplification #2: reduced to 8098 unique words

---- simplification #3: reduced to 7158 unique words

---- simplification #4: reduced to 7092 unique words

---- simplification #5: reduced to 7074 unique words

---- simplification #6: reduced to 7071 unique words

---- simplification #7: reduced to 7071 unique words

DONE!

Compressed to 10.79% of the original vocabulary

Total time: 1.28s

Here are a few of the 7071 unique words:

['', 'ZSHSHSHT', 'SHTHTHZHST', 'SHZHSTS', 'HTHSHTSHS', 'STZHSTHT', 'HZSHS', 'STHZSHT', 'HSHTHTHTST', 'THSHTHZHST', 'HTHSTHTST', 'TSTSHZ', 'SHZTHS', 'THSTZH', 'HTHTSTHSH', 'ZHTHT', 'STHTHSTHSTH', 'HZSTHZH', 'HTZSHTHS', 'SHTHSTHTHZ']

From these, 198 are Cliffords:

['', 'HZSHS', 'ZSZH', 'SHSZS', 'HZHZH', 'ZSHZS', 'HSHSZHS', 'SHSHSZ', 'HSHSHZS', 'HZSHZS', 'SHZ', 'SZHZS', 'HSZSH', 'ZHSZH', 'HSHZSHS', 'ZSHSHSH', 'HSZHZ', 'SZSHZ', 'HSHSZH', 'H

In [70]:
# ######## memory inneficient implementation (stores the entire vocab before simplifying)

# letters = ['H','T']
# L = 15
# maxnsimp = 100

# start = time.time()

# vocab = []
# simp = {}
# for l in range(1,L+1):
#     vocab = vocab + vocabulary(letters, l)
#     for k,v in simplification_rules(l).items():
#         simp[k] = v
# totalsize = len(vocab)
# size = totalsize # will be updated

# print(f'There are {size} words in the vocabulary with length up to {L}')

# for i in range(1,maxnsimp):

#     vocab = [list(run_length.encode(word)) for word in vocab]
#     vocab = [' '.join([k[0]+str(k[1]) for k in word]) for word in vocab]
#     # print(f'Vocabulary using run length encoding:\n{vocab}\n')
#     vocab = [word.split(' ') for word in vocab]
#     vocab = [[(simp[syl] if syl != '' else '') for syl in word] for word in vocab]
#     vocab = list(set([''.join(word) for word in vocab]))
#     print(f'\n---- simplification #{i}: reduced to {len(vocab)} unique words')
#     if len(vocab) < size:
#         size = len(vocab)
#     else:
#         print(f'\nDONE!\n\nCompressed to {round(100*size/totalsize,2)}% of the original vocabulary\n')
#         break

# print(f'Here are some of the unique words:\n\n{vocab[:50]}')

# end = time.time()
# print(f'\nTotal time: {round(end-start,2)}s')